# FIDE Data on player ranking
an in-depth analysis

In [1]:
import pandas as pd

df = pd.read_xml("players_list_xml_foa.xml")
df.head(10)

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag
0,25941186,None,IND,M,None,None,None,None,NaN,NaN,NaN,1009.0,5.0,40.0,NaN,NaN,NaN,NaN,None
1,5855560,None,SGP,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,1808060,None,WLS,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,21533636,",alfaifi,Wael ali qasim",KSA,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1995.0,None
4,10292519,"A A M Imtiaz, Chowdhury",BAN,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975.0,None
5,10688862,"A Abdel Maabod, Hoda",EGY,F,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009.0,w
6,33496722,A Aman,IND,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,None
7,35853913,"A Aziz, Mohd Azizi Jamil",MAS,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981.0,None
8,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1977.0,None
9,10245154,"A B M Jobair, Hossain",BAN,M,None,None,None,None,1583.0,0.0,40.0,1597.0,5.0,20.0,1625.0,0.0,20.0,1998.0,None


In [5]:
df.head(35)

,fideid,name,country,sex,title,w_title,o_title,foa_title,rating,games,k,rapid_rating,rapid_games,rapid_k,blitz_rating,blitz_games,blitz_k,birthday,flag
0,25941186,None,IND,M,None,None,None,None,NaN,NaN,NaN,1009.0,5.0,40.0,NaN,NaN,NaN,NaN,None
1,5855560,None,SGP,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,1808060,None,WLS,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,21533636,",alfaifi,Wael ali qasim",KSA,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1995.0,None
4,10292519,"A A M Imtiaz, Chowdhury",BAN,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975.0,None
5,10688862,"A Abdel Maabod, Hoda",EGY,F,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009.0,w
6,33496722,A Aman,IND,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,None
7,35853913,"A Aziz, Mohd Azizi Jamil",MAS,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981.0,None
8,10224084,"A B M Hasibuzzaman, Tapan",BAN,M,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1977.0,None
9,10245154,"A B M Jobair, Hossain",BAN,M,None,None,None,None,1583.0,0.0,40.0,1597.0,5.0,20.0,1625.0,0.0,20.0,1998.0,None
